In [ ]:

#Import libaries

#For reading data from cvc
import pandas as pd

#Calculation
import numpy as np

#Calculation
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors


#For KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.cluster.vq import kmeans, vq,whiten
from sklearn.metrics import silhouette_score

#For faster computation(?)
from numba import jit 

#For randomize
import random

#For Mapping
from geopandas import GeoDataFrame
import seaborn as sns
from shapely.geometry import Point
import geopandas as gpd

import folium

In [ ]:
#Supress warnings for jit
import logging;
logger = logging.getLogger("numba")
logger.setLevel(logging.ERROR)

In [ ]:
#Get data with pd
"""Take in the file"""
df = pd.read_csv('US_Accidents_May19_Migrated Data.csv')
df.head()

In [ ]:
city = df.loc[df['City'] == "New York"]

print(city["Start_Lat"[:20]])
print(city["End_Lng"[:20]])

In [ ]:
#Acess wanted column and convert to numpy
latitude = np.array(df["Start_Lat"])
longitude = np.array(df["Start_Lng"])


In [ ]:
#To see if latitude and longitude match in case some data are lost or NaN
@jit(nopython = True)
def match() -> bool:
    if (len(latitude) == len(longitude)):
        return True
    else:
        return False
    
print(match())

In [ ]:
#To get easy acess of the data
#np.savetxt(r'lng.txt', longitude, fmt='%1.6f')
#np.savetxt(r'lat.txt', latitude, fmt='%1.6f')

In [ ]:
#Combining the latitude og longitude together
@jit 
def comb(lat : list ,long : list) -> list:
    data = np.column_stack((lat,long))
    return data

data = comb(latitude,longitude)

In [ ]:
#Just in case, the coordinates are saved to a file
#np.savetxt(r'Points.txt', data, fmt='%1.6f') #uncomment to save as txt

Beacuse the data are over 2 millions for each axis, the analysis will be taking a random sample of 100 000 of them.

In [ ]:
#Choosing random points with seed(10)
def random_sample() -> list:
    l = list(range(2243939))
    random.seed(10)
    sorted_list = sorted(random.sample(l, 100000))
    return sorted_list

pick = np.array(random_sample())

In [ ]:
#Create an subset from the random sample
#@jit
def data_sub_set(random_list : list , data_set : list) -> list:
    #ramdom_list is the random index we will pick from
    #data_set is which data points we will pick from
    sub_set = np.empty([len(random_list),2])
    iterator = 0
    for i in random_list:
        sub_set[iterator] = data_set[i]
        iterator += 1
    return sub_set

new_set = data_sub_set(pick,data)

In [ ]:
#To see how the random data is spread out on a grid
new_set = data_sub_set(pick,data)
new_lat = np.array([row[0] for row in new_set])
new_lng = np.array([row[1] for row in new_set])


plt.scatter(new_lat ,new_lng,s=4)
plt.grid(True)
plt.show()

In [ ]:
#To Calculate number of optimal k means
#This function shows how many cluster point we should have. 
def optimise_k_means(data, max_k): 
  means = []
  inertias = []
  
  for k in range(1,max_k):
      kmeans = KMeans(n_clusters = k) 
      kmeans.fit(data) 
      
      means.append(k)
      inertias.append(kmeans.inertia_)
      
  #Generate plot
  fig = plt.subplots(figsize = (10,5))
  plt.plot(means,inertias, "o-")
  plt.xlabel("Numbers of Clusters")
  plt.ylabel("Inertia")
  plt.grid(True)
  plt.show()
  
optimise_k_means(new_set,10)

By looking at the plot we can see that the number of cluster is 3 Another method to confirm this is to use silhoutte method.

In [ ]:
#Calculate silhoutte_method
def silhoutte_method(data, max_k):
    sil = [None,None]
    
    for k in range(2,max_k+1):
        kmeans = KMeans(n_clusters = k).fit(data)
        labels = kmeans.labels_
        
        sil.append(silhouette_score(data, labels, metric = 'euclidean'))
    return sil
    

In [ ]:
"""This does not work. Takes to long???"""
#Generate plot 
# fig = plt.subplots(figsize = (10,5))
# plt.plot(silhoutte_method(new_set,10), 10)
# plt.xlabel("Numbers of Clusters")
# plt.ylabel("silhouette score")
# plt.grid(True)
# plt.show()


In [ ]:
#Generete the k_mean cluster plot
#Making the cluster plot 
def create_cluster_plot(numb_cluster : int , data):
    x_pt = np.array([row[0] for row in data])
    y_pt = np.array([row[1] for row in data])
    kmeans = KMeans(n_clusters = numb_cluster)
    kmeans.fit(data)
    kmeans_label = kmeans.labels_
    
    plt.scatter(x = x_pt, y= y_pt , c = kmeans_label)
    plt.show()


create_cluster_plot(3,new_set)

In [ ]:
#Tried to make a subplot, but does not work
def comparing_n_cluster(max_cluster : int, data):

    list_of_means = []
    x_pt = np.array([row[0] for row in new_set])
    y_pt = np.array([row[1] for row in data])
    for k in range(1,max_cluster):
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(data)
        kmeans_label = kmeans.labels_
        list_of_means.append(kmeans_label)
    
    #fig, axs = plt.subplots(nrows=1, ncols=5, figsize=(20,5))
    
    ist_of_means = list(list_of_means)
    
    #for i, ax in enumerate(fig.axes,start=1):
    #    ax.scatter(x = np.array(row[0] for row in data), y = np.array(row[1] for row in data))
    for i in range(1,max_cluster):
        plt.scatter(x = x_pt, y = y_pt , c = list_of_means[i])
        plt.show()
    
        
comparing_n_cluster(6, new_set)

In [ ]:
#For comparing different cluster
for i in range(2,6):
    create_cluster_plot(i,new_set)


In [ ]:
lat = np.array([row[0] for row in new_set])
long = np.array([row[1] for row in new_set])

latitude = 38.500000
longitude = -95.665


map_clusters = folium.Map(location=[latitude,longitude], zoom_start=4)

kmeans = KMeans(n_clusters = 3)
kmeans.fit(new_set)
kmeans_label = kmeans.labels_

# set color scheme for the clusters
x = np.arange(4)
ys = [i + x + (i*x)**2 for i in range(4)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng,cluster, in zip(lat,long,kmeans_label):
    #label = folium.Popup(str(city)+ ','+str(state) + '- Cluster ' + str(cluster), parse_html=True)
    folium.vector_layers.CircleMarker(
        [lat, lng],
        radius=5,
        #popup=label,
        #tooltip = str("New York")+ ',' +str(state) + ' - Cluster ' + str(cluster),
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters)
       
map_clusters